In [14]:
# create a simple database where store the relation <r_id; s_id; SLOTH_overlap>
import psycopg
import psycopg.rows


class ResultDatabase:
        
    def __init__(self, dbname, table_name='results_table'):
        self.dbname = dbname
        self.table_name = table_name
        self._dbconn = psycopg.connect(f"port=5442 host=/tmp dbname={self.dbname}", row_factory=psycopg.rows.dict_row)

        q = f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_name   = '{self.table_name}'
            );
        """

        exists = self._dbconn.execute(q).fetchone()['exists'] == True
        if not exists:
            self._dbconn.execute(
                f"""
                CREATE TABLE {table_name} (
                    r_id integer NOT NULL,
                    s_id integer NOT NULL,
                    sloth_overlap integer NOT NULL,
                    PRIMARY KEY (r_id, s_id)
                );

                CREATE INDEX {table_name}_r_id_index ON {table_name}(r_id);
                """
            )
        else:
            print('Results table already exists')

        self._dbconn.commit()

    def insert_results(self, values:list[list[int,int,int]]):
        self._dbconn.cursor().executemany(f"INSERT INTO {self.table_name} VALUES(%s, %s, %s)", values)
        self._dbconn.commit()

    def lookup_result_table(self, r_id, s_id):
        """ Where the r_id < s_id """

        result = self._dbconn.execute(
            f"""
            SELECT sloth_overlap FROM {self.table_name}
            WHERE r_id = {r_id} AND s_id = {s_id}
            """
        ).fetchone()

        return None if len(result) == 0 else result['sloth_overlap']
        
    def clear(self):
        self._dbconn.execute(f"TRUNCATE {self.table_name};")
        self._dbconn.commit()

    def close(self):
        self._dbconn.close()


rdb = ResultDatabase('nanni')
rdb.insert_results([[111, 222, 6], [111, 555, 32], [333, 777, 9]])

print(rdb.lookup_result_table(111, 555))
rdb.close()



Results table already exists
Results table already exists
32
